In [1]:
import os
import glob
import time
from copy import deepcopy

import warnings

import pandas as pd
import numpy as np

from cobra.io import read_sbml_model, write_sbml_model, load_json_model
from cobra import Model, Reaction, Metabolite

import Simulator

/data/gbkim/anaconda3/envs/ibridge/lib/python2.7/site-packages/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")


## Module 1

In [2]:
def run_MetScore_simulation(filename, biomass_rxn, target_rxn, constdic={}):
    simulator_obj = Simulator.Simulator()
    simulator_obj.read_model(filename)
    a,b,flux_dic = simulator_obj.run_FBA(internal_flux_minimization=True)
    wild_opt_flux = flux_dic
    print('#constraints', constdic)
    
    _, _, flux_dic = simulator_obj.run_FBA(new_objective=target_rxn, 
                                         flux_constraints = constdic, 
                                         internal_flux_minimization=True, 
                                         mode='max')
    max_const = flux_dic[target_rxn]
     
    _, _, flux_dic = simulator_obj.run_FBA(new_objective=target_rxn, 
                                           flux_constraints=constdic, 
                                           internal_flux_minimization=True, 
                                           mode='min')    
    min_const = flux_dic[target_rxn]    

    flux_dist_dic_set = {}   
    count = 0  
    for each_flux_const in np.linspace(min_const, max_const, 10):
        tmp_const = deepcopy(constdic)
        count+=1
        tmp_const[target_rxn] = [each_flux_const*0.95, each_flux_const*1.05]
        stat, obj_val, opt_flux_dic = simulator_obj.run_FBA(new_objective=biomass_rxn, 
                                                            flux_constraints=tmp_const, 
                                                            mode='max')
        
        if stat != 2:
            print('Biomass optimization failed at count %d'%count)
            continue
        
        tmp_biomass_flux = opt_flux_dic[biomass_rxn]
        tmp_const[biomass_rxn] = [tmp_biomass_flux*0.95, tmp_biomass_flux*1.05]
        
        print('Count: %d\tBiomass flux: %0.6f'%(count, tmp_biomass_flux))
        
        stat, obj_val, flux_dic = simulator_obj.run_MOMA(wild_flux=wild_opt_flux,
                                                         flux_constraints=tmp_const)

        if stat == 2:
            flux_dist_dic_set[each_flux_const] = flux_dic
            
        '''CHECK'''
        if abs(flux_dic[biomass_rxn]) <= 1e-6: 
            break

    flux_df = pd.DataFrame.from_dict(flux_dist_dic_set)
    flux_corr_df = flux_df.abs().T.corr()
    flux_cov_df = flux_df.abs().T.cov()
    return flux_df, flux_corr_df, flux_cov_df


def calculate_MetScore_sum(cobra_model, covariance_data):
    branch_metabolite_data = {}
    neighboring_rxns = {}

    for each_metabolite in cobra_model.metabolites:
        target_met_id = each_metabolite.id
        branch_metabolite_data[target_met_id] = 0.0
        count = 0
        for each_reaction in each_metabolite.reactions:
            reactants = [met.id for met in each_reaction.reactants]
            if target_met_id in reactants:
                if each_reaction.id in covariance_data:
                    branch_metabolite_data[target_met_id] += abs(covariance_data[each_reaction.id])
    
    return branch_metabolite_data


def select_candidates(cobra_model, target_reaction, output_file, 
                      metscore_df, corr_df, cov_df, 
                      corr_threshold=0, cov_threshold=0.1):
    pcorr_df = corr_df[corr_df > corr_threshold]
    pcov_df = cov_df[cov_df > cov_threshold]
    pcorr_df = pcorr_df.dropna()
    pcov_df = pcov_df.dropna()
    
    positive_candidate_reactions = list(set(pcorr_df.index)&set(pcov_df.index))
    
    ncorr_df = corr_df[corr_df < -corr_threshold]
    ncov_df = cov_df[cov_df < -cov_threshold]
    ncorr_df = ncorr_df.dropna()
    ncov_df = ncov_df.dropna()
    
    negative_candidate_reactions = list(set(ncorr_df.index)&set(ncov_df.index))
    
    print('Number of positive candidate reactions: %d'%(len(positive_candidate_reactions)))
    print('Number of negative candidate reactions: %d'%(len(negative_candidate_reactions)))
    
    fp = open(output_file, 'w')
    header = ['Metabolite', 'Score', 'Normalized score', 
              'No. of reactions', 'No. of positive reactions', 
              'No. of negative reactions', 'candidate reactions', 
              'positive reactions', 'negative reactions', 
              'Positive score', 'Negative score']
    fp.write('%s\n'%('\t'.join(header)))
    for each_row, each_df in metscore_df.iterrows():
        cobra_metabolite = cobra_model.metabolites.get_by_id(each_row)
        candidate_reactions = []

        for each_reaction in cobra_metabolite.reactions:
            for each_reactant in each_reaction.reactants:
                if each_reactant.id == each_row:
                    candidate_reactions.append(each_reaction.id)
        
        candidate_reactions = list(set(candidate_reactions))   
        pos_candidate_reactions = list(set(positive_candidate_reactions)&set(candidate_reactions))
        neg_candidate_reactions = list(set(negative_candidate_reactions)&set(candidate_reactions))
        
        positive_score = 0.0
        negative_score = 0.0
        for rxn in pos_candidate_reactions:
            positive_score+=float(pcov_df.loc[rxn])
        for rxn in neg_candidate_reactions:
            negative_score+=float(ncov_df.loc[rxn])
        if each_df[target_reaction] != 0.0 and np.sqrt(float(len(candidate_reactions))) != 0.0:
            normalized_score = each_df[target_reaction]/np.sqrt(float(len(candidate_reactions)))
        else:
            normalized_score = 0.0
        
        contents = [each_row, each_df[target_reaction], normalized_score,
                    len(candidate_reactions), len(pos_candidate_reactions),
                    len(neg_candidate_reactions), ';'.join(candidate_reactions),
                    ';'.join(pos_candidate_reactions), ';'.join(neg_candidate_reactions),
                    positive_score, negative_score]
        for i, item in enumerate(contents):
            if i < len(contents)-1:
                delim = '\t'
            else:
                delim = '\n'
            if type(item) == str:
                fp.write(item + delim)
            else:
                fp.write(str(item) + delim)
    
    fp.close()
    return


def read_target_rxn(filename):
    fp = open(filename, 'r')
    lines = fp.read()
    lines = lines.replace('[', '')
    lines = lines.replace(']', '')
    lines = lines.replace("'", "")
    fp.close()
    target_rxn = lines.strip().split(',')
    print(len(target_rxn))
    return target_rxn
    

def write_header(filename, taget_id):
    fp = open(filename, 'r')
    lines = fp.read()
    fp.close()
    fp = open(filename, 'w')
    fp.write('%s,%s\n'%('reaction', taget_id))
    fp.write(lines.strip())
    fp.close()
    return

## Module 2

In [3]:
def make_candidate_reaction_sets(df, basename, input_dir, output_dir):
    ex_metabolites = []
    with open('%s/excluded_metabolites.txt'%input_dir, 'r') as fp:
        for line in fp:
            ex_metabolites.append(line.strip())
            
    score_info = {}
    for each_met, each_df in df.groupby('Metabolite'):
            
        if each_met[-1] == 'c':
            pos_reaction_num = each_df['No. of positive reactions'].values[0]
            neg_reaction_num = each_df['No. of negative reactions'].values[0]

            pos_score = each_df['Positive score'].values[0]
            neg_scroe = each_df['Negative score'].values[0]
            
            final_pos_scroe = pos_score/np.sqrt(1+pos_reaction_num)
            final_neg_scroe = neg_scroe/np.sqrt(1+neg_reaction_num)
            score_info[each_met] = [final_pos_scroe, final_neg_scroe]

    negative_score_info = {}
    positive_score_info = {}
    
    for met in score_info:
        if abs(score_info[met][0]) > abs(score_info[met][1]):
            positive_score_info[met] = score_info[met][0]
        else:
            negative_score_info[met] = score_info[met][1]
      
    if not os.path.exists('%s/application_results'%output_dir):
        os.makedirs('%s/application_results'%output_dir)
    fp = open('%s/application_results/Candidate_%s'%(output_dir, basename), 'w')
    fp.write('%s\t%s\t%s\t%s\n'\
             %('Negative metabolite', 'Positive metabolite',
               'Negative score', 'Positive score'))
    for negative_met in negative_score_info:
        negative_score = negative_score_info[negative_met]
        
        for positive_met in positive_score_info:
            
            if negative_met in ex_metabolites or positive_met in ex_metabolites:
                continue
            
            positive_score = positive_score_info[positive_met]
            fp.write('%s\t%s\t%s\t%s\n'\
                     %(negative_met, positive_met, negative_score, positive_score))
            
    fp.close()
    return
    
    
def check_reaction(filename, original_model_met_info, universal_model_met_info, flux_corr_df, flux_cov_df, output_dir):
    basename = os.path.basename(filename)
    ex_reaction_id = basename.split('Final_MetScore_')[1].strip()
    ex_reaction_id = ex_reaction_id.replace('.txt', '')
    
    df = pd.read_table(filename)
    if not os.path.exists('%s/application_result2'%output_dir):
        os.makedirs('%s/application_result2'%output_dir)
    fp = open('%s/application_result2/New_reaction_candidate_up_%s'%(output_dir, basename), 'w')
    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s'\
             %('Target', 'Gene source', 'Negative metabolite', 
              'Positive metabolite', 'Negative score', 'Positive score', 
              'Reaction', 'Equation', 'Corr', 'Cov'))
    
    for each_row, each_df in df.iterrows():
        negative_met = each_df['Negative metabolite']
        positive_met = each_df['Positive metabolite']

        negative_score = each_df['Negative score']
        positive_score = each_df['Positive score']
        
        for each_met_set in original_model_met_info:
            if negative_met in each_met_set[0] and positive_met in each_met_set[1]:
                target_reaction = each_met_set[2]
                if target_reaction not in flux_corr_df.index:
                    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
                             %(ex_reaction_id, 'Native', negative_met, positive_met, 
                              negative_score, positive_score, each_met_set[2], 
                              each_met_set[3], 'NA', 'NA'))
                else:
                    corr_val = float(flux_corr_df.loc[target_reaction])
                    cov_val = float(flux_cov_df.loc[target_reaction])
                    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
                             %(ex_reaction_id, 'Native', negative_met, positive_met, 
                              negative_score, positive_score, each_met_set[2], 
                              each_met_set[3], corr_val, cov_val))
                    
        
        for each_met_set in universal_model_met_info:
            if negative_met+'_' in each_met_set[0] and positive_met+'_' in each_met_set[1]:
                target_reaction = each_met_set[2]
                if target_reaction not in flux_corr_df.index:
                    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
                             %(ex_reaction_id, 'Universal model', negative_met, positive_met, 
                              negative_score, positive_score, each_met_set[2], 
                              each_met_set[3], 'NA', 'NA'))
    fp.close()
    return


def check_down_reaction(filename, original_model_met_info, universal_model_met_info, flux_corr_df, flux_cov_df, output_dir):
    basename = os.path.basename(filename)
    ex_reaction_id = basename.split('Final_MetScore_')[1].strip()
    ex_reaction_id = ex_reaction_id.replace('.txt', '')
    
    df = pd.read_table(filename)
    if not os.path.exists('%s/application_result2'%output_dir):
        os.makedirs('%s/application_result2'%output_dir)
    fp = open('%s/application_result2/New_reaction_candidate_down_%s'%(output_dir, basename), 'w')
    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s'\
             %('Target', 'Gene source', 'Negative metabolite', 
              'Positive metabolite', 'Negative score', 'Positive score', 
              'Reaction', 'Equation', 'Corr', 'Cov'))
    
    for each_row, each_df in df.iterrows():
        negative_met = each_df['Negative metabolite']
        positive_met = each_df['Positive metabolite']

        negative_score = each_df['Negative score']
        positive_score = each_df['Positive score']
        
        for each_met_set in original_model_met_info:
            if negative_met in each_met_set[1] and positive_met in each_met_set[0]:
                target_reaction = each_met_set[2]
                if target_reaction not in flux_corr_df.index:
                    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
                             %(ex_reaction_id, 'Native_down', negative_met, positive_met, 
                              negative_score, positive_score, each_met_set[2], 
                              each_met_set[3], 'NA', 'NA'))
                else:
                    corr_val = float(flux_corr_df.loc[target_reaction])
                    cov_val = float(flux_cov_df.loc[target_reaction])
                    fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
                             %(ex_reaction_id, 'Native_down', negative_met, positive_met, 
                              negative_score, positive_score, each_met_set[2], 
                              each_met_set[3], corr_val, cov_val))
    fp.close()
    return





def metabolite_set(cobra_model):
    met_set_info = []
    for each_reaction in cobra_model.reactions:
        reactants = [met.id for met in each_reaction.reactants]
        products = [met.id for met in each_reaction.products]
        met_set_info.append([reactants, products, each_reaction.id, each_reaction.reaction])
    return met_set_info


def metabolite_set_cytosol(cobra_model):
    met_set_info = []
    for each_reaction in cobra_model.reactions:
        reactants = [met.id for met in each_reaction.reactants]
        products = [met.id for met in each_reaction.products]

        compartments = []
        for each_met in reactants+products:
            each_cmp = each_met[-2:]
            compartments.append(each_cmp)
            
        compartments = list(set(compartments))
        if len(compartments) == 1:
            if compartments[0] == 'c_':
                met_set_info.append([reactants, products, each_reaction.id, each_reaction.reaction])
        
    return met_set_info

## Module 3

In [4]:
def parse_reactions(reaction, ex_metabolites):
    reaction = reaction.replace('>', '')
    reaction = reaction.replace('<', '')
    
    sptlist = reaction.split('--')
    reactants = sptlist[0].strip().split(' + ')
    products = sptlist[0].strip().split(' + ')
    new_reactants = []
    new_products = []
    
    for met in reactants:
        if met not in ex_metabolites:
            new_reactants.append(met)
            
    for met in products:
        if met not in ex_metabolites:
            new_products.append(met)
            
    return new_reactants, new_products

def check_duplicate(reaction, known_reaction_info, ex_metabolites):
    flag = False
    reactants, products = parse_reactions(reaction, ex_metabolites)
    if len(reactants) > 0 and len(products) > 0: 
        for rxn in known_reaction_info:
            reactants2 = known_reaction_info[rxn][0]
            products2 = known_reaction_info[rxn][1]

            if  set(reactants).issubset(set(reactants2)) and set(products).issubset(set(products2)):
                flag = True
                return flag

            if  set(products).issubset(set(reactants2)) and set(reactants).issubset(set(products2)):
                flag = True
                return flag
        
    return flag

## Inputs

In [5]:
model_file = './input/ijo1366_IRR_indirubin.xml'
original_model_file = './input/ijo1366_IRR.xml'
universal_model_file = './input/bigg_universal_model.xml'
biomass_rxn = 'Ec_biomass_iJO1366_core_53p95M'
target_rxn = 'EX_INDIRUBIN_LPAREN_e_RPAREN_'


input_dir = './input'
output_dir = './results'

ko_reactions = ['PYK']


target_rxn_list = [target_rxn]

#Strain phenotype
constrict={}
for item in ko_reactions:
    constrict[item] = [0.0, 0.0]

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Read cobra models

In [6]:
warnings.filterwarnings(action='ignore')

if not glob.glob(universal_model_file):
    tmp_universal_model = load_json_model("./input/universal_model.json")
    write_sbml_model(tmp_universal_model, universal_model_file, use_fbc_package=False)

cobra_model = read_sbml_model(model_file)
original_cobra_model = read_sbml_model(original_model_file)
universal_cobra_model = read_sbml_model(universal_model_file)

warnings.filterwarnings(action='default')

## First module

In [7]:
for target_reaction in target_rxn_list:
    print('Target reaction: %s'%target_reaction)
    model_reactions = [reaction.id for reaction in cobra_model.reactions]

    corr_output = '%s/corr_%s.csv'%(output_dir,target_reaction)
    cov_output = '%s/cov_%s.csv'%(output_dir, target_reaction)

    flux_df, flux_corr_df, flux_cov_df = run_MetScore_simulation(model_file, biomass_rxn, 
                                                                 target_reaction, constrict)
    flux_corr_df = flux_corr_df[target_reaction].loc[model_reactions]
    flux_corr_df.to_csv(corr_output)
    flux_cov_df = flux_cov_df[target_reaction].loc[model_reactions]
    flux_cov_df.to_csv(cov_output)

    write_header(corr_output, target_reaction)
    write_header(cov_output, target_reaction)

    df = pd.read_csv(cov_output, index_col=0, header=0)

    final_dic = {}
    for each_col in df.columns:
        final_dic[each_col]={}
        fluxsum_dic = calculate_MetScore_sum(cobra_model,  dict(df[each_col]))
        final_dic[each_col]=fluxsum_dic

    final_df = pd.DataFrame.from_dict(final_dic)
    final_df.to_csv('%s/MetScore_%s.csv'%(output_dir, target_reaction))

    flux_corr_df = pd.read_csv(corr_output, index_col=0)
    flux_cov_df = pd.read_csv(cov_output, index_col=0)
    output_file = '%s/Final_MetScore_%s.txt'%(output_dir, target_reaction)
    select_candidates(cobra_model, target_reaction, output_file, 
                      final_df, flux_corr_df, flux_cov_df, 0, 0.0)

Target reaction: EX_INDIRUBIN_LPAREN_e_RPAREN_
Using license file /data/gbkim//gurobi.lic
Academic license - for non-commercial use only
('#constraints', {'PYK': [0.0, 0.0]})
Count: 1	Biomass flux: 0.982372
Count: 2	Biomass flux: 0.880322
Count: 3	Biomass flux: 0.777387
Count: 4	Biomass flux: 0.674451
Count: 5	Biomass flux: 0.571516
Count: 6	Biomass flux: 0.468582
Count: 7	Biomass flux: 0.365647
Count: 8	Biomass flux: 0.262711
Count: 9	Biomass flux: 0.158953
Count: 10	Biomass flux: 0.051092
Number of positive candidate reactions: 174
Number of negative candidate reactions: 483


# Second module

In [8]:
files = glob.glob('%s/Final_*.txt'%output_dir)
for filename in files:
    basename = os.path.basename(filename)
    df = pd.read_table(filename)
    make_candidate_reaction_sets(df, basename, input_dir, output_dir)

In [9]:
original_model_met_set_info = metabolite_set(original_cobra_model)
universal_model_met_set_info = metabolite_set_cytosol(universal_cobra_model)
print('Number of metabolite set for original model: %d'\
          %len(original_model_met_set_info))
print('Number of metabolite set in cytosol for universal model: %d'\
          %len(universal_model_met_set_info))

files = glob.glob('%s/application_results/*.txt'%output_dir)

cnt = 1
for each_file in files:
    s = time.time()
    basename = os.path.basename(each_file)
    ex_reaction_id = basename.split('Final_MetScore_')[1].strip()
    ex_reaction_id = ex_reaction_id.replace('.txt', '')

    corr_file = glob.glob('%s/corr*%s*.csv'%(output_dir, ex_reaction_id))[0]
    cov_file = glob.glob('%s/cov*%s*.csv'%(output_dir, ex_reaction_id))[0]

    flux_corr_df = pd.read_csv(corr_file, index_col=0)
    flux_cov_df = pd.read_csv(cov_file, index_col=0)

    print('Analysis number: %d\tTarget reaction: %s'%(cnt, ex_reaction_id))

    check_reaction(each_file, original_model_met_set_info, universal_model_met_set_info, flux_corr_df, flux_cov_df, output_dir)
    check_down_reaction(each_file, original_model_met_set_info, universal_model_met_set_info, flux_corr_df, flux_cov_df, output_dir)
    cnt+=1
    e = time.time()
    print('Elapsed time: %fs'%(e-s))

Number of metabolite set for original model: 3193
Number of metabolite set in cytosol for universal model: 11948
Analysis number: 1	Target reaction: EX_INDIRUBIN_LPAREN_e_RPAREN_
Elapsed time: 151.907056s


## Third module

In [23]:
ex_metabolites = []
with open('%s/excluded_metabolites.txt'%input_dir, 'r') as fp:
    for line in fp:
        ex_metabolites.append(line.strip())

known_reaction_info = {}
for each_reaction in original_cobra_model.reactions:
    reactants, products = parse_reactions(each_reaction.reaction, ex_metabolites)
    known_reaction_info[each_reaction] = [reactants, products]
    
        

files = [item for item in os.listdir('%s/application_result2/'%output_dir) if '_up_' in item]
fp = open('%s/Application_final_up_summary.txt'%output_dir, 'w')
fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
         %('Target', 'Gene source', 'Negative metabolite', 
          'Positive metabolite', 'Negative score', 'Positive score', 
          'Reaction', 'Equation', 'Corr', 'Cov'))

for each_file in files:
    with open('%s/application_result2/%s'%(output_dir, each_file), 'r') as fp2:
        fp2.readline()
        for line in fp2:
            sptlist = line.strip().split('\t')
            equation = sptlist[7].strip()
            met1 = sptlist[2].strip()
            met2 = sptlist[3].strip()
            if met1 not in ex_metabolites and met2 not in ex_metabolites:
                fp.write(line.strip() + '\n')
fp.close()


files = [item for item in os.listdir('%s/application_result2/'%output_dir) if '_down_' in item]
fp = open('%s/Application_final_down_summary.txt'%output_dir, 'w')
fp.write('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'\
         %('Target', 'Gene source', 'Negative metabolite', 
          'Positive metabolite', 'Negative score', 'Positive score', 
          'Reaction', 'Equation', 'Corr', 'Cov'))

for each_file in files:
    with open('%s/application_result2/%s'%(output_dir, each_file), 'r') as fp2:
        fp2.readline()
        for line in fp2:
            sptlist = line.strip().split('\t')
            equation = sptlist[7].strip()
            met1 = sptlist[2].strip()
            met2 = sptlist[3].strip()
            if met1 not in ex_metabolites and met2 not in ex_metabolites:
                fp.write(line.strip() + '\n')
fp.close()